# Synthetic Dialogue Generation

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/00_overview/1.single_llm_full_generation.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>


## Getting started

### Environment Setup

Let's first check if our environment is all set up.

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    # Downloading only the "output" directory from the repository
    !git init .
    !git remote add -f origin https://github.com/Play-Your-Part/tutorials.git
    !git config core.sparseCheckout true
    !echo "output" >> .git/info/sparse-checkout
    !git pull origin main

    # Installing Ollama
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


> ⚠️ If you're using **Colab**, please, **restart the runtime** once everything above is installed

### Ollama Setup

Let's run the ollama server first, as a background process:

In [2]:
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 10  # Let's wait a bit for the server to start

0

Let's change the default sdialog model to `gemma3:12b`:

In [3]:
import sdialog

sdialog.config.llm("ollama:gemma3:12b")

### Defining the Output (Dialogue)

We will begin by defining the JSON objects that we will use to represent the generated dialogues. For now this object will have only three fields: `"model"`, `"seed"`, `"scenario"`, and `"dialog"` to store the name of the model and the seed used to generate the dialogue, as well as the scenario associated to the dialogue and the dialogue itself, respectively. More preciselly, the `"dialog"` field will contain the list of turns of the conversation in order, with the speaker name and the corresponding utterances. As shown in the following example:

In [4]:
example_dialogue = {
    "model": "qwen2.5:14b",  # the model used to generate the dialogue
    "seed": 123,  # the seed used to generated
    "scenario": "short hello and good bye conversation",  # the scenario used to generated the dialogue
    "turns": [
        {"speaker": "Alice", "text": "Hey Bob!"},
        {"speaker": "Bob", "text": "Hey Alice!"},
        {"speaker": "Alice", "text": "Bye Bob!"},
        {"speaker": "Bob", "text": "Bye bye!"},
    ]
}

We can use `pydantic` to properly define our `Dialogue` type:

In [5]:
from pydantic import BaseModel
from typing import List, Union, Optional

class Turn(BaseModel):
    speaker: str
    text: str

class Dialog(BaseModel):
    model: str  # the model used to generate the dialogue
    seed: int  # the seed used to generated
    scenario: Optional[Union[dict, str]] = None  # the scenario used to generated the dialogue
    turns: List[Turn]  # the list of turns of the conversation

Having a Python `pydantic` class to formally represent our dialogues is quite useful, we can convert any JSON dialogue to our `Dialog` class as follows:

In [6]:
my_dialogue = Dialog.model_validate(example_dialogue)
my_dialogue

Dialog(model='qwen2.5:14b', seed=123, scenario='short hello and good bye conversation', turns=[Turn(speaker='Alice', text='Hey Bob!'), Turn(speaker='Bob', text='Hey Alice!'), Turn(speaker='Alice', text='Bye Bob!'), Turn(speaker='Bob', text='Bye bye!')])

Or the opposite, convert our `Dialog`s to a `dict` or a JSON as follows:

In [7]:
my_dialogue.model_dump()  # a dict

{'model': 'qwen2.5:14b',
 'seed': 123,
 'scenario': 'short hello and good bye conversation',
 'turns': [{'speaker': 'Alice', 'text': 'Hey Bob!'},
  {'speaker': 'Bob', 'text': 'Hey Alice!'},
  {'speaker': 'Alice', 'text': 'Bye Bob!'},
  {'speaker': 'Bob', 'text': 'Bye bye!'}]}

In [8]:
my_dialogue_json = my_dialogue.model_dump_json(indent=2)  # a string containing the dialog as a JSON object
print(my_dialogue_json)

{
  "model": "qwen2.5:14b",
  "seed": 123,
  "scenario": "short hello and good bye conversation",
  "turns": [
    {
      "speaker": "Alice",
      "text": "Hey Bob!"
    },
    {
      "speaker": "Bob",
      "text": "Hey Alice!"
    },
    {
      "speaker": "Alice",
      "text": "Bye Bob!"
    },
    {
      "speaker": "Bob",
      "text": "Bye bye!"
    }
  ]
}


Or, of course, create a new `Dialog` from scratch:

In [9]:
Dialog(
    model="qwen2.5:14b",
    seed=123,
    turns=[
        Turn(speaker="Alice", text="Hi :)"),
        Turn(speaker="Bob", text="Bye! :(")
    ]
)

Dialog(model='qwen2.5:14b', seed=123, scenario=None, turns=[Turn(speaker='Alice', text='Hi :)'), Turn(speaker='Bob', text='Bye! :(')])

Alternativelly, we can use the built-in `Dialog` class from `sdialog`:

In [10]:
from sdialog import Dialog

my_dialog = Dialog.model_validate(example_dialogue)
my_dialog

Dialog(version='0.4.1+fdf16e10134b2ac7dea23518390cf5b09aa10380', timestamp='2025-11-21T10:12:08Z', model='qwen2.5:14b', seed=123, id='97c70f30-7a3e-4c6e-a7d8-4b0ca2e7d767', parentId=None, complete=None, personas=None, context=None, scenario='short hello and good bye conversation', turns=[Turn(speaker='Alice', text='Hey Bob!'), Turn(speaker='Bob', text='Hey Alice!'), Turn(speaker='Alice', text='Bye Bob!'), Turn(speaker='Bob', text='Bye bye!')], events=None, notes=None)

Which besides providing the exact same functionalities, among other things, allow us to:

- Pretty print the dialogue:

In [11]:
my_dialog.print()

[dialog_id] 97c70f30-7a3e-4c6e-a7d8-4b0ca2e7d767
[model] qwen2.5:14b
[seed] 123
--- Dialogue Begins ---
[Alice] Hey Bob!
[Bob] Hey Alice!
[Alice] Bye Bob!
[Bob] Bye bye!
--- Dialogue Ends ---


- Print it in a vanilla textual form:

In [12]:
print(my_dialog)

Alice: Hey Bob!
Bob: Hey Alice!
Alice: Bye Bob!
Bob: Bye bye!


- Save it to a file:

In [13]:
# either as a JSON object
my_dialog.to_file("output/my_dialogue.json")

# or a txt file
my_dialog.to_file("output/my_dialogue.txt")

_(check created files [`output/my_dialogue.json`](output/my_dialogue.json) and [`output/my_dialogue.txt`](output/my_dialogue.txt))_

- Load a dialogue from disk

In [14]:
my_dialog = Dialog.from_file("output/my_dialogue.json")
my_dialog.print(scenario=True)  # `scenario=True` to also print the metadata stored in scenario field

[dialog_id] 97c70f30-7a3e-4c6e-a7d8-4b0ca2e7d767
[model] qwen2.5:14b
[seed] 123
[scenario] 
short hello and good bye conversation
--- Dialogue Begins ---
[Alice] Hey Bob!
[Bob] Hey Alice!
[Alice] Bye Bob!
[Bob] Bye bye!
--- Dialogue Ends ---


In [15]:
my_dialog = Dialog.from_file("output/my_dialogue.txt")
my_dialog.print(scenario=True)

[dialog_id] 262292f0-b751-4774-9598-877f33558a86
--- Dialogue Begins ---
[Alice] Hey Bob!
[Bob] Hey Alice!
[Alice] Bye Bob!
[Bob] Bye bye!
--- Dialogue Ends ---


- Or simple things like quickly know how long a dialogue is:

In [16]:
len(my_dialog)

4

Now we are ready to begin working on synthetic `Dialog` (;)) generation!

## Dialogue Generation

### Description-driven Generation

We can use the `sdialog`'s built-in `DialogueGenerator` class that we can instantiate using descriptions to generate our dialogues:

In [17]:
from sdialog.generators import DialogGenerator

Which takes the following arguments as input:
- `model` model name to use (any model tag from [ollama hub](https://ollama.com/library)).
- `dialogue_details` the details about the desired dialogue.
- `output_format` the output format as a `pydantic` class or JSON scheme, as we did above (`LLMDialogOutput` by default).
- `scenario` an optional metadata field that describes the scenario used to generated dialogue.

For instance, let's create an instance of `DialogGenerator` to generate conversations between Bob and Alice about her birthday:

In [18]:
dialog_generator = DialogGenerator(
    dialogue_details="The conversation is between a dad (Bob) and his doughter (Alice). "
                     "Her birthday is coming up and she wants to throw a Star Wars themed party.",
)

[2025-11-21 11:12:08] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Then, we can use the build-in `.generate()` method to generate conversations for such instance:

_(each time you run the code below, a different dialogue will be generated)_

In [19]:
dialog = dialog_generator.generate()
dialog.print(scenario=True)

[dialog_id] cd02e639-d4b8-4f47-ab06-e125c0a43735
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'top_p': 0.95, 'temperature': 1, 'top_k': 64}
[seed] 296780219
--- Dialogue Begins ---
[Alice] Hey Dad! Guess what?
[Bob] Hey sweetie! What's got you so excited?
[Alice] My birthday is getting closer! And I was thinking... I really, *really* want to have a Star Wars themed party!
[Bob] A Star Wars party? Wow, that's ambitious! Sounds like fun, though. You’re a big Star Wars fan, aren't you?
[Alice] The biggest! I want everything to be Star Wars! Decorations, games, even the food!
[Bob] Okay, okay! Let's not get carried away just yet. A Star Wars party is going to take some planning. What did you have in mind?
[Alice] I was thinking we could have lightsaber training! And a scavenger hunt for droid parts! And maybe a Death Star piñata?
[Bob] A Death Star piñata? That’s brilliant! Lightsaber training sounds like a bit of a challenge for us adults, but we'll figure something out. What about f

Let's now change the description, now the party has to be about Lord of the Rings, not Star Wars. To do this we can simply pass the new description when calling the generate method:

In [20]:
dialog = dialog_generator.generate(
    "The conversation is between a dad (Bob) and his doughter (Alice). "
    "Her birthday is coming up and she wants to throw a Lord of the Rings themed party."
)
dialog.print(scenario=True)

[dialog_id] be322ef8-011d-4013-9029-7883b9d65bdb
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'top_p': 0.95, 'temperature': 1, 'top_k': 64}
[seed] 1364604708
--- Dialogue Begins ---
[Alice] Hey Dad! Guess what?
[Bob] Hey sweetie! What’s got you so excited?
[Alice] My birthday is getting closer, and I was thinking… I really want to have a Lord of the Rings party!
[Bob] A Lord of the Rings party? Wow, that's ambitious! That's a lot more than just balloons and cake.
[Alice] I know! But I’m so excited about it! We could decorate like the Shire, and maybe even have a costume contest!
[Bob] A costume contest? Okay, that sounds pretty fun. But, are you sure you have enough friends who would be into dressing up as hobbits or elves?
[Alice] Pretty much everyone in my class loves Lord of the Rings! We watch the movies all the time. And I’m already brainstorming costume ideas!
[Bob] Alright, alright, you're convincing me. What about food? Lembas bread isn't exactly easy to bake.
[Alice] Haha

You can use the `seed` number above to re-generate the exact same dialogue each time as an argument of `generate()`, as follows:

In [21]:
dialog_generator.generate(
        "The conversation is between a dad (Bob) and his doughter (Alice). "
        "Her birthday is coming up and she wants to throw a Lord of the Rings themed party.",
        seed=1364604708
).print()

[dialog_id] 62070161-0671-499e-9c0f-0448ac085160
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'top_p': 0.95, 'temperature': 1, 'top_k': 64}
[seed] 1364604708
--- Dialogue Begins ---
[Alice] Hey Dad! Guess what?
[Bob] Hey sweetie! What’s got you so excited?
[Alice] My birthday is getting closer, and I was thinking… I really want to have a Lord of the Rings party!
[Bob] A Lord of the Rings party? Wow, that's ambitious! That's a lot more than just balloons and cake.
[Alice] I know! But I’m so excited about it! We could decorate like the Shire, and maybe even have a costume contest!
[Bob] A costume contest? Okay, that sounds pretty fun. But, are you sure you have enough friends who would be into dressing up as hobbits or elves?
[Alice] Pretty much everyone in my class loves Lord of the Rings! We watch the movies all the time. And I’m already brainstorming costume ideas!
[Bob] Alright, alright, you're convincing me. What about food? Lembas bread isn't exactly easy to bake.
[Alice] Haha

### Role-Playing-based Generation

Our goal here will be to have to LLM to generate the dialogues by role-playing the different charecters.

Each character will be fully defined by its persona, so, the same way started this tutorial by defining what a "synthetic `Dialog`" will actually be, we should now define our `Persona`.

Hopefully, the `sdialog` contains a `BasePersona` that we can import to create our own custom persona classes, let's import it:

In [22]:
from sdialog.personas import BasePersona

And let's define our concrete `Persona` class now by specifying some useful attributes like a name, role, background, etc:

In [23]:
class Persona(BasePersona):
    name: str = ""
    role: str = ""
    background: str = ""
    personality: str = ""
    circumstances: str = ""
    rules: str = ""

Now we can create/instantiate any `Persona` we want for our characters. Let's create our Bob and Alice:

In [24]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

If we print `bob` we will see that it is automatically converted to natural language description, which is usefull when we want to create the actual prompt for our LLM.

In [27]:
print(bob_persona)

* Name: Bob
* Role: great dad
* Background: Computer Science PhD.
* Personality: an extremely happy person that likes to help people
* Circumstances: Your daughter will talk to you


We can also pretty print it with `.print()` method, as with dialog objects:

In [ ]:
bob_persona.print()

--- Persona Begins ---
[Name] Bob
[Role] great dad
[Background] Computer Science PhD.
[Personality] an extremely happy person that likes to help people
[Circumstances] Your daughter will talk to you
[Rules] 
--- Persona Ends ---


In case we are working with a really complex persona and this default description is not good for your needs, you can overwrite it by defining your own `description()` method as in the following example:

In [28]:
class PersonaCustom(BasePersona):
    name: str = ""
    role: str = ""

    def description(self):
        return f"Your awesome name is {self.name} and your awesome role is being a {self.role}"

awesome_bob = PersonaCustom(
        name="Bob",
        role="great dad"
)

# Let's print "awesome_bob" persona
print(awesome_bob)

Your awesome name is Bob and your awesome role is being a great dad


So, we now know how to create personas, let's move to the fun part which is actually creating the actual generator.

Fortunatelly, we can simply use `sdialog`'s built-in `PersonaDialogGenerator` class to generate our persona-based dialogues as follows:

In [29]:
from sdialog.generators import PersonaDialogGenerator

dialog_generator = PersonaDialogGenerator(
    persona_a=bob_persona,
    persona_b=alice_persona,
)

dialog_generator.generate().print()

[2025-11-21 11:15:35] INFO:sdialog.util:Loading Ollama model: gemma3:12b


[dialog_id] 2442f84c-00f1-4e14-a0ae-5c7b56cbbe80
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3259483948
--- Dialogue Begins ---
[SPEAKER_B] Hi Dad!
[SPEAKER_A] Alice! Hi sweetie! How are you doing today?
[SPEAKER_B] I’m great! I was wondering if we could talk about my birthday party?
[SPEAKER_A] Absolutely! I'm all ears. What's on your mind?
[SPEAKER_B] I really, really want to have a Lord of the Rings theme!
[SPEAKER_A] A Lord of the Rings party! That's fantastic! That's such a brilliant idea, I love it! We can really do something special. What were you thinking?
[SPEAKER_B] I thought we could decorate like the Shire! And maybe have a costume contest?
[SPEAKER_A] The Shire decorations are perfect! And a costume contest? Oh, that's going to be a riot! I can already picture it - little hobbits running around!
[SPEAKER_B] Yeah! And maybe a 'quest' for the kids to complete?
[SPEAKER_A] A quest! Brilliant! We can hide little 'rings

## Use Case: Dialogue Generation for STAR Dataset

### Introduction

Before we begin this section, make sure you have the STAR dataset downloaded in your system, inside the `datasets` folder:

In [ ]:
# Let's clone the STAR dataset repository
!git clone https://github.com/RasaHQ/STAR.git datasets/STAR

# Let's check that `dialogues` and `tasks` folders are inside `datasets/STAR`
!ls datasets/STAR

The [STAR](https://arxiv.org/pdf/2010.11853) dataset contains 6652 human-generated dialogues as JSON objects where files are named as `NUMBER.json`.

Humans had to follow a well-defined set of instruction to generate the dialogue role playing the system (wizard) and the client (user).

For instance, clicking [here](datasets/STAR/dialogues/1.json) we can open the file [`1.json`](datasets/STAR/dialogues/1.json) containing the first dialogue. For now, let's focus only on the `"Scenario"` field.

For instance, for the dialogue in `1.json` it is as follows:

```json
{
    "Domains": [  # List of domains
        "doctor"
    ],
    "Happy": true,  # Wheather or not the dialogue follos a happy path
    "MultiTask": false,  # Wheather or not this dialogue involves more than one task
    "UserTask": "You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.",
    "WizardTask": "Inform the user of his/her doctor's orders.",
    "WizardCapabilities": [  # List of flowcharts describing the each task the Wizard is cable of doing
        {
        "Domain": "doctor",
        "SchemaImage": "doctor_followup.jpg",
        "Task": "doctor_followup"
        }
    ]
}
```

We can use the `STAR` module from `sdialog` to read scenarios object from any dialogue in STAR given it's id given a STAR conversation id as follows:

In [31]:
from sdialog.datasets import STAR

# Let's first indicate where the dataset is located
STAR.set_path("datasets/STAR/")

# Let's set the first dialogue as the target example
TARGET_DIALOG = 1

# Let's load the scenario of the first dialog
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

{'Domains': ['doctor'],
 'Happy': True,
 'MultiTask': False,
 'UserTask': 'You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.',
 'WizardCapabilities': [{'Domain': 'doctor',
   'SchemaImage': 'doctor_followup.jpg',
   'Task': 'doctor_followup'}],
 'WizardTask': "Inform the user of his/her doctor's orders."}

Which corresponds to the following dialogue:

In [32]:
original_dialogue = STAR.get_dialog(1)
original_dialogue.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


### Description-driven Generation

In the `scenario`, we can see that in this conversation, the user's behavior is defined by instructions given in natural language (`"UserTask"`), however, the system/wizard behavior is more rigidly defined as a graph describing the dialogue policy to followed (since system was expected to be more deterministic). These graphs are described as JSON objects storing the graph edges as key:value pairs (source:destination). We can find these graphs in the [`STAR/tasks`](datasets/STAR/tasks) folder.

Ideally, we would like our `DialogGenerator` to generate dialogues for each different scenario. That is, given a `scenario` we would like generate multiple dialogues for it.

To achieve this, we only need to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Fortunately, we can use the built-in `get_scenario_description()` method to do this, which takes an `scenario` as input and returns its natural language description containing all the details (including the system behavior described by the graphs):

In [33]:
print(STAR.get_scenario_description(scenario))

The conversation is between a User and a AI assistant in the following domains: doctor.

The User instructions are: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.
The AI assistant instructions are: Inform the user of his/her doctor's orders.

In addition, the AI assistant is instructed to follow specific flowcharts to address the tasks.
Flowcharts are defined as graph described using DOT.
The actual DOT for the current tasks are:

The graph for the task 'doctor_followup' with domain 'doctor' is:
```dot
digraph doctor_followup  {
    hello -> ask_name;
    ask_name -> doctor_ask_doctor_name;
    doctor_ask_doctor_name -> query;
    query -> doctor_inform_doctors_instructions;
    doctor_inform_doctors_instructions -> anything_else
}
```
and one example responses for each node is provided in the following json:
```json
{
  "hello": "H

Note how the original graph in JSON describing the system's behavior have been converted to a [DOT](https://en.wikipedia.org/wiki/DOT_(graph_description_language)) description which should be easier to interpret by the LLM, since DOT is a well-known format to describe graphs in plain text. 

Let's now put these two methods together and create a function that given a STAR dialogue ID will generate a natural language description of the scenario associated to it, simply as follows:

In [34]:
def get_dialog_scenario_description(dialogue_id):
    # Get the scenario of the target dialogue
    scenario = STAR.get_dialog_scenario(dialogue_id)
    # Then return it along its description in natural language
    return scenario, STAR.get_scenario_description(scenario)

With this function now we have everything we need to generate synthethic dialogues for STAR that follows the same scenario as a given target real dialogue.

For instance, let's say we want to generate dialogues following the same scenario as the first STAR dialogue, we can simply:

In [35]:
# First, let's get the scenario and description of the first dialogue
scenario, description = get_dialog_scenario_description(dialogue_id=1)

# le'ts now create a dialogue generator for it
dialog_generator = DialogGenerator(
    dialogue_details=description,
    scenario=scenario
)

[2025-11-21 11:16:11] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Let's now generate multiple conversation that follows the same scenario as dialogue 1 of STAR dataset.
> **Note**
> Run the cell multiple times to get different conversations for it

In [36]:
dialog = dialog_generator.generate()
dialog.print(scenario=True)

[dialog_id] ab87f4ce-95ac-4470-b62d-27a90bb1d22a
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'top_p': 0.95, 'temperature': 1, 'top_k': 64}
[seed] 2131212703
[scenario] 
{
  "Domains": [
    "doctor"
  ],
  "Happy": true,
  "MultiTask": false,
  "UserTask": "You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.",
  "WizardCapabilities": [
    {
      "Domain": "doctor",
      "SchemaImage": "doctor_followup.jpg",
      "Task": "doctor_followup"
    }
  ],
  "WizardTask": "Inform the user of his/her doctor's orders."
}
--- Dialogue Begins ---
[AI assistant] Hello, how can I help?
[Alexis] Hi, I had an appointment with Dr. Morgan recently and I unfortunately forgot to write down the instructions she gave me. I need to know how often to take my medicine.
[AI assistant] Could I get your name, please?
[Alexis] It's Alexis.
[AI assistant] W

We can see the LLM is able to follow the scenario surprisingly well, specially for the system part which is guided by a graph with pre-defined responses.

Now update the generator to match a more challenging scenario, let's say that of dialogue 5100 that is multi-task and does not follow a happy path:

In [37]:
scenario, description = get_dialog_scenario_description(5100)

dialog_generator = DialogGenerator(
    dialogue_details=description,
    scenario=scenario
)
scenario

[2025-11-21 11:16:20] INFO:sdialog.util:Loading Ollama model: gemma3:12b


{'Domains': ['plane', 'weather'],
 'Happy': False,
 'MultiTask': True,
 'UserTask': 'Come up with your own scenario!\n\nAbout you:\n- Your name: Ben\n\n The AI Assistant can handle:\n- Search for a flight (e.g. from Chicago to Pittsburgh)\n- Book a flight (e.g. with id 193)\n- Checking the weather forecast in different Cities (e.g. Chicago or Pittsburgh)',
 'WizardCapabilities': [{'Domain': 'plane',
   'SchemaImage': 'plane_search.jpg',
   'Task': 'plane_search'},
  {'Domain': 'plane', 'SchemaImage': 'plane_book.jpg', 'Task': 'plane_book'},
  {'Domain': 'weather', 'SchemaImage': 'weather.jpg', 'Task': 'weather'}],
 'WizardTask': 'Follow the flow charts and help the user.'}

And generate dialogues for it:

_(run multi-times the call to generate different ones)_

In [38]:
dialog_generator.generate().print()

[dialog_id] 8d7211d0-4cfe-44ed-bdc2-dfd12ed7b6d6
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 637994336
--- Dialogue Begins ---
[Ben] Hello, how can I help?
[User] Hi Ben, I'm Sarah. I'm calling about a flight and then the weather.
[Ben] May I have your name, please?
[User] It's Sarah, as I just said.
[Ben] Where are you departing from?
[User] Chicago.
[Ben] Where are you going to?
[User] Pittsburgh.
[Ben] On what day do you want the plane to arrive?
[User] Next Friday, please.
[Ben] I can also filter the results for a specific airline, class, price or duration.
[User] No, just give me the cheapest option. And now, can you check the weather in Pittsburgh for the same day?
[Ben] Right, I found CLAZZ flight with AIRLINE for PRICE credits that takes DURATION hours to get to ARRIVAL_CITY.
[User] Actually, forget the flight. It’s too expensive. Can you tell me the weather?
[Ben] For what day would you like the weather forecast?
[User

### Role-playing-based Generation

Before, in previous section we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user personas for each scenario which means we have to return the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` of the first dialogue above:

In [39]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)

user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

* Language: English
* Role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
* Circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.


Now, similar to what we did in the previous subsection, we just need to define a function that given a dialogue ID can return its scenario as well as the system and user persona for it: 

In [40]:
def get_dialog_scenario_and_personas(dialogue_id):
    # Get the scenario of the target dialogue
    scenario = STAR.get_dialog_scenario(dialogue_id)
    # Get the personas
    system = STAR.get_system_persona_for_scenario(scenario)
    user = STAR.get_user_persona_for_scenario(scenario)
    return scenario, system, user

And that's it, not we can simply create a `PersonaDialogGenerator` using the system and user personas as follows:

In [41]:
# let's get the personas and the scenario
scenario, system, user = get_dialog_scenario_and_personas(dialogue_id=1)

# le'ts now create a dialogue generator for it
dialog_generator = PersonaDialogGenerator(
    persona_a=system,
    persona_b=user,
    scenario=scenario
)

# let's generate the dialogue
dialog_generator.generate().print()

[2025-11-21 11:16:39] INFO:sdialog.util:Loading Ollama model: gemma3:12b


[dialog_id] 84838b85-328a-420e-ac9c-0c62f5deeef8
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'top_k': 64, 'top_p': 0.95, 'temperature': 1}
[seed] 983680493
--- Dialogue Begins ---
[SPEAKER_A] Hello, how can I help?
[SPEAKER_B] Hi! I need to know what instructions Dr. Morgan gave me during my appointment. I forgot to write them down.
[SPEAKER_A] Could I get your name, please?
[SPEAKER_B] It's Alexis.
[SPEAKER_A] Who is your doctor?
[SPEAKER_B] Dr. Morgan.
[SPEAKER_A] Your instructions are: Take one tablet twice a day, with food. Specifically, one tablet at 8 AM and one tablet at 6 PM. It is important to adhere to this schedule for best results.
[SPEAKER_B] Oh, that's perfect! Thank you so much!
[SPEAKER_A] Is there anything else that I can do for you?
[SPEAKER_B] No, that's all. Thanks again!
[SPEAKER_A] Thank you and goodbye.
--- Dialogue Ends ---


And that's it for this tutorial, congrats! 😎

### Saving our dialogues

Finally, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

Let's first get all happy dialogues for this task using `sdialog`'s built-in `STAR.get_dialogs()` function:

In [42]:
original_dialogs = STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False)
print('Total number of happy "doctor_followup" dialogues in STAR:', len(original_dialogs))

Total number of happy "doctor_followup" dialogues in STAR: 105


Now let's generate the dialogues and save them in the path pointed by the `PATH_OUTPUT` variable.

In [43]:
import os

from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/full-generation"

path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(original_dialogs, desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.id}.txt")):
        continue

    scenario, description = STAR.get_dialog_scenario_description(dialog.id)
    dialog_generator = DialogGenerator(
        dialogue_details=description,
        scenario=scenario
    )
    dialog = dialog_generator.generate(id=dialog.id, seed=dialog.id)

    # Normalize speaker names in each turn (since their also generated by the LLM)
    for turn in dialog.turns:
        turn.speaker = "System" if "AI" in turn.speaker else "User"

    dialog.to_file(os.path.join(path_json, f"{dialog.id}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.id}.txt"))

Finally, let's check the files were generated:

In [46]:
%ls output/STAR/full-generation/

In [ ]:
%ls output/STAR/full-generation/txt

Cool, huh? Congrats for finalizing the tutorial! you did a great job! 😎